In [ ]:
#imports
import numpy as np
from matplotlib import pyplot as plt
from javelin.structure import Structure
from javelin.unitcell  import UnitCell
from javelin.fourier   import Fourier 
from IPython.display import clear_output
from javelin.utils import run_parallel
import javelin.BvsOptimizer as bvs
import Scatty
import BvsOpt 

In [ ]:
%matplotlib qt
plt.style.use(['science'])
plt.rcParams['text.usetex'] = False
plt.rcParams['axes.linewidth'] = 2
plt.rcParams["font.family"] = "Arimo"
plt.rcParams['xtick.major.size'] = 4
plt.rcParams['xtick.major.width'] = 3
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 2
plt.rcParams['ytick.major.size'] = 4
plt.rcParams['ytick.major.width'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.minor.width'] = 2
seshadri = ['#c3121e', '#0348a1', '#ffb01c', '#027608', '#0193b0', '#9c5300', '#949c01', '#7104b5']

In [ ]:
x = np.linspace(0.8, 3, 100)
y = 100*(1/x)**12
plt.figure(figsize = [8,6])
plt.plot(x,y, color = "green", ls="-", lw = 4)
plt.xlabel("Distance")
plt.ylabel("Potential")

In [ ]:
def lc(x): 
    return 4.3548 - 0.1743*x

def CreateStructure(ncells = 10, doping = 0): 
    pos = [(0,0,0),(0.5,0,0),(0,0.5,0),(0,0,0.5),(0.5,0.5,0.5)]
    cubic_cell = UnitCell(lc(doping))
    BaK = np.random.choice([19,56], size =ncells**3 , p=[doping, 1-doping])
    structure = Structure(numbers=[83,8,8,8,1]*ncells**3,
    positions = pos*ncells**3, unitcell=cubic_cell)
    structure.reindex([ncells,ncells,ncells,5])
    structure.get_atomic_numbers()[4::5] = BaK
    structure.update_atom_symbols()
    return structure

In [ ]:
xs = np.linspace(0.2,0.5,100)
ys = lc(xs)
plt.figure(figsize = [8,6])
plt.plot(xs,ys*np.sqrt(2)/2, ls = "-", lw=3)
plt.hlines([1.26 + 1.52], xmin=0.2, xmax = 0.5)
plt.xlabel("Doping", fontsize=16)
plt.ylabel("Lattice constant (A)", fontsize = 16)

In [ ]:
def Yukawa(structure = None): 
    str = bvs.Structure_to_Bvs(structure)
    doping = structure.get_average_site(4)['K']['occ']
    BiO =  structure.get_neighbors(0,1) + structure.get_neighbors(0,2) + structure.get_neighbors(0,3)  
    BaO =  structure.get_neighbors(4,1) + structure.get_neighbors(4,2) + structure.get_neighbors(4,3)

    Range = 2
    distance = 3*Range

    BaBi = structure.get_neighbors(4,0, maxD = distance)
    BiBa = structure.get_neighbors(0,4, maxD = distance)
    BiBi = structure.get_neighbors(0,0, maxD = distance)
    BaBa = structure.get_neighbors(4,4, maxD = distance)

    
    Bi_bonds = structure.get_neighbors(0,1) + structure.get_neighbors(0,2) + structure.get_neighbors(0,3)
    Ba_bonds = structure.get_neighbors(4,1) +structure.get_neighbors(4,2)+structure.get_neighbors(4,3)
    O1_bonds = structure.get_neighbors(1,0) + structure.get_neighbors(1,4)
    O2_bonds = structure.get_neighbors(2,0) + structure.get_neighbors(2,4)
    O3_bonds = structure.get_neighbors(3,0) + structure.get_neighbors(3,4)

    str.set_bonds(0, Bi_bonds.values)
    str.set_bonds(4, Ba_bonds.values)
    str.set_bonds(1, O1_bonds.values)
    str.set_bonds(2, O2_bonds.values)
    str.set_bonds(3, O3_bonds.values)

    str.set_parameters(83,8,2.050,0.318)
    str.set_parameters(56,8,2.223,0.406)
    str.set_parameters(19,8,2.047,0.398)

    str.set_nominals_and_weights([83,56,19,8] ,[4+doping,2,1,2],[0,0,0,0], [1,1,1,1] , [1,1,1,0])

    sigma_Bi = 0.03
    sigma_Ba = 0.06


    mod_Bi  = BvsOpt.Generator(0, [0,0,0] ,[sigma_Bi, sigma_Bi, sigma_Bi])
    mod_Ba  = BvsOpt.Generator(4, [0.5,0.5,0.5], [sigma_Ba, sigma_Ba, sigma_Ba])

    av1 = np.sqrt(3)/2*lc(doping)
    K1 = 1
    K2 = 1e-2
    f = 10
    Range = Range*lc(doping)
    nb1 = BvsOpt.Calculator(0, (BiBa + BiBi + BiO).values)
    nb1.add_interaction(83,56,"Yukawa", [f*(4+doping),f*2, Range])
    nb1.add_interaction(83,19,"Yukawa", [f*(4+doping),f*1, Range])
    nb1.add_interaction(83,83,"Yukawa", [f*(4+doping),f*(4+doping), Range])
    nb1.add_interaction(83,8, "Morse", [K1, 0.5*lc(doping),0.3])

    
    nb2 = BvsOpt.Calculator(4, (BaBi + BaBa + BaO).values)
    nb2.add_interaction(56,83,"Yukawa", [f*2, f*(4+doping), Range])
    nb2.add_interaction(19,83,"Yukawa", [f*1, f*(4+doping), Range])
    nb2.add_interaction(19,19,"Yukawa", [f*1, f*1, Range])
    nb2.add_interaction(19,56,"Yukawa", [f*1, f*2, Range])
    nb2.add_interaction(56,56,"Yukawa", [f*2, f*2, Range])
    nb2.add_interaction(56,8, "Morse", [K2, lc(doping)*np.sqrt(2)/2,0.4])
    nb2.add_interaction(19,8, "Morse", [K2, lc(doping)*np.sqrt(2)/2,0.4])


    sim = BvsOpt.Simulation()
    sim.add_generator(mod_Bi)
    sim.add_generator(mod_Ba)

    sim.add_calculator(nb1)
    sim.add_calculator(nb2)

    sim.optimize_valence = False
    sim.optimize_vector_valence = False

    BaBi = structure.get_neighbors(4,0)
    BiBa = structure.get_neighbors(0,4)
    BiBi = structure.get_neighbors(0,0)
    BaBa = structure.get_neighbors(4,4)


    cycles = 300
    for i in range(cycles):
        clear_output(wait=True)
        print("Cycle: ", i)
        sim.run_cycle(str, 1, 1, 1e-1)
        print("Bi_rms: ", str.rms_displacement(0), "Ba_rms: ", str.rms_displacement(4))
        print("Bi-Ba: ", (str.average_distance(83,56,BiBa.values) - av1)/lc(doping) , "Bi-K: ", (str.average_distance(83,19,BiBa.values) - av1)/lc(doping))
    
    return bvs.Bvs_to_Structure(str , cell = lc(doping))





In [ ]:
#Bragg peaks
structure_av = CreateStructure(ncells = 30, doping=0.4)
Bragg = Fourier() 
Bragg.approximate = True
Bragg.average = False
Bragg.lots = 10,10,10
Bragg.number_of_lots = 50
Bragg.radiation = "xray"
Bragg.grid.r1 = -8,8
Bragg.grid.r2 = -8,8
Bragg.grid.bins = (801, 801, 1) 

bragg = Bragg.calc_average(structure_av)

In [ ]:
f1 = Fourier() 
f1.approximate = True
f1.average = True
f1.lots = 4,4,4
f1.number_of_lots = 40
f1.radiation = "xray"
f1.grid.r1 = -8,8
f1.grid.r2 = -8,8
f1.grid.bins = (801, 801, 1) 

I = [f1.calc(x) for x in structures]
diffuse_intensity = sum(I)

In [ ]:
%matplotlib qt
plt.style.use(['science'])
plt.rcParams['text.usetex'] = False


In [ ]:
(2*diffuse_intensity+ bragg/10).rename("Intensity").plot(figsize = [12,10], cmap='magma_r', shading = "auto",  vmax = 4.5*10**8)


In [ ]:
def BvsOptimization(structure = None): 
    str = bvs.Structure_to_Bvs(structure)
    doping = 1-structure.get_average_site(4)['Ba']['occ']
    Bi_bonds = structure.get_neighbors(0,1) + structure.get_neighbors(0,2) + structure.get_neighbors(0,3)
    Ba_bonds = structure.get_neighbors(4,1) +structure.get_neighbors(4,2)+ structure.get_neighbors(4,3)
    O1_bonds = structure.get_neighbors(1,0) + structure.get_neighbors(1,4)
    O2_bonds = structure.get_neighbors(2,0) + structure.get_neighbors(2,4)
    O3_bonds = structure.get_neighbors(3,0) + structure.get_neighbors(3,4)

    OO_1 = structure.get_neighbors(1,2) + structure.get_neighbors(1,3) 
    OO_2 = structure.get_neighbors(2,1) + structure.get_neighbors(2,3) 
    OO_3 = structure.get_neighbors(3,1) + structure.get_neighbors(3,2) 


    str.set_bonds(0, Bi_bonds.values)
    str.set_bonds(4, Ba_bonds.values)
    str.set_bonds(1, O1_bonds.values)
    str.set_bonds(2, O2_bonds.values)
    str.set_bonds(3, O3_bonds.values)

    str.set_parameters(83,8,2.050,0.318)
    str.set_parameters(56,8,2.223,0.406)
    str.set_parameters(19,8,2.047,0.398)

    str.set_nominals_and_weights([83,56,19,8] ,[4+doping,2,1,2],[0,0,0,0], [1,1,1,1] , [1,1,1,1])
    sigma_Bi = 0.02
    sigma_Ba = 0.01
    sigma_O =  0.06
    

    mod_Bi  = BvsOpt.Generator(0, [0,0,0] ,[sigma_Bi, sigma_Bi, sigma_Bi])
    mod_Ba  = BvsOpt.Generator(4, [0.5,0.5,0.5], [sigma_Ba, sigma_Ba, sigma_Ba])
    mod_1  =  BvsOpt.Generator(1, [0.5,0,0] ,    [0, sigma_O, sigma_O])
    mod_2  =  BvsOpt.Generator(2, [0,0.5,0] ,    [sigma_O, 0, sigma_O])
    mod_3  =  BvsOpt.Generator(3, [0,0,0.5] ,    [sigma_O, sigma_O, 0])


    
    D = 600
    R_BaO = np.sqrt(2)*lc(doping)/2 - 0.1*lc(doping)
    R_KO =  np.sqrt(2)*lc(0.45)/2
    R_BiO = 0
    K = 1000
    d = lc(doping)*np.sqrt(2)/2


    calc1 = BvsOpt.Calculator(0, (Bi_bonds).values)
    calc1.add_interaction(83,8,"SoftVolumeExclusion", [D, R_BiO])

    calc2 = BvsOpt.Calculator(4, (Ba_bonds).values)
    calc2.add_interaction(56,8,"SoftVolumeExclusion", [D, R_BaO])
    calc2.add_interaction(19,8,"SoftVolumeExclusion", [D, R_KO])
    
    
    calc3 = BvsOpt.Calculator(1, (O1_bonds + OO_1).values)
    calc3.add_interaction(8,56,"SoftVolumeExclusion", [D, R_BaO])
    calc3.add_interaction(8,19,"SoftVolumeExclusion", [D, R_KO])
    calc3.add_interaction(8,83,"SoftVolumeExclusion", [D, R_BiO]) 
    calc3.add_interaction(8,8,"Spring", [K, d])

    calc4 = BvsOpt.Calculator(2, (O2_bonds + OO_2).values)
    calc4.add_interaction(8,56,"SoftVolumeExclusion", [D, R_BaO])
    calc4.add_interaction(8,19,"SoftVolumeExclusion", [D, R_KO])
    calc4.add_interaction(8,83,"SoftVolumeExclusion", [D, R_BiO]) 
    calc4.add_interaction(8,8,"Spring", [K, d])


    calc5 = BvsOpt.Calculator(3, (O3_bonds + OO_3).values)
    calc5.add_interaction(8,56,"SoftVolumeExclusion", [D, R_BaO])
    calc5.add_interaction(8,19,"SoftVolumeExclusion", [D, R_KO])
    calc5.add_interaction(8,83,"SoftVolumeExclusion", [D, R_BiO]) 
    calc5.add_interaction(8,8,"Spring", [K, d])


    sim = BvsOpt.Simulation()
    sim.add_generator(mod_1)
    sim.add_generator(mod_2)
    sim.add_generator(mod_3)
    sim.add_calculator(calc3)
    sim.add_calculator(calc4)
    sim.add_calculator(calc5)


    sim.optimize_vector_valence = False
    sim.optimize_valence =  True

    av1 = lc(doping)*np.sqrt(3)/2
    av2 = lc(doping)*np.sqrt(2)/2

    cycles = 200

    for i in range(cycles):
        #T = 20*np.exp(-i/40) + 1
        T=1
        clear_output(wait=True)
        print("Cycle: ", i) 
        sim.run_cycle(str, 0.005 ,1, T)
        print("GII:", str.global_instability_index())
        print("O rms: ", str.rms_displacement(1))
        print("Ba_val:", str.get_average_valence(56), "K_val: ", str.get_average_valence(19), "Bi_val: ", str.get_average_valence(83))
        print("Ba-O: ",  (str.average_distance(56,8, Ba_bonds.values) - av2)/lc(doping), "K-O: ", (str.average_distance(19,8,Ba_bonds.values) - av2)/lc(doping))
        print("O-O: ",  str.average_distance(8,8, (OO_1 + OO_2 + OO_3).values)/lc(doping)) 

        cs = str.CS_parameter(Bi_bonds.values)
        print("CS_parameter:", cs)
        print("P: ", str.local_polarization(Bi_bonds.values))
 
    return bvs.Bvs_to_Structure(str , cell = lc(doping))

In [ ]:
structure = CreateStructure(15, 0.5)


In [ ]:
structure = BvsOptimization(structure)

In [ ]:
xs = []
p = []
csp = []

In [ ]:
Bi_bonds = structure.get_neighbors(0,1) + structure.get_neighbors(0,2) + structure.get_neighbors(0,3)

x = 0.51
structures=[CreateStructure(15,x) for i in range(6)] 
structures=run_parallel(BvsOptimization, structures)
structures = [bvs.Structure_to_Bvs(str) for str in structures]
ps = [str.local_polarization(Bi_bonds.values) for str in structures]
csps=[str.CS_parameter(Bi_bonds.values) for str in structures]
xs.append(x)
p.append(sum(ps)/len(ps))
csp.append(sum(csps)/len(csps))
    
    

In [ ]:
plt.figure(figsize = [8,4])
plt.plot(xs, 100*csp,   color=seshadri[3], marker='o', linestyle='-', mew=2, lw = 2, mfc='w', alpha=.8, markersize=8)
plt.legend(fontsize = 14) 
plt.xlabel("Doping", fontsize = 12)
plt.ylabel("Centrosymmetry parameter (arb. units)", fontsize = 12)

In [ ]:
Bi_bonds = structure.get_neighbors(0,1) + structure.get_neighbors(0,2) + structure.get_neighbors(0,3)
Ba_bonds = structure.get_neighbors(4,1) + structure.get_neighbors(4,2)+structure.get_neighbors(4,3)
O1_bonds = structure.get_neighbors(1,0) + structure.get_neighbors(1,4)
O2_bonds = structure.get_neighbors(2,0) + structure.get_neighbors(2,4)
O3_bonds = structure.get_neighbors(3,0) + structure.get_neighbors(3,4)
OO_1 = structure.get_neighbors(1,2) + structure.get_neighbors(1,3) 
OO_2 = structure.get_neighbors(2,1) + structure.get_neighbors(2,3) 
OO_3 = structure.get_neighbors(3,1) + structure.get_neighbors(3,2) 
dop1 = structure.get_average_site(4)['K']['occ']
list_BaO =  bvs.get_distances(structure, atom1 = 56, atom2 = 8, neighborlist= Ba_bonds)/lc(dop1)
list_KO=   bvs.get_distances (structure, atom1 = 19, atom2 = 8, neighborlist= Ba_bonds)/lc(dop1)
list_OO=   bvs.get_distances (structure, atom1 = 8, atom2 = 8, neighborlist= OO_1+OO_2+OO_3)/lc(dop1)



In [ ]:
plt.figure(figsize=[8,11])
plt.hist(list_BaO  , bins = 90 , density = True, color = seshadri[3], fill = False,label="Ba-O pairs",  histtype = "step", lw = 3, zorder= 2, alpha=.7)
plt.hist(list_KO , bins = 90 , density = True, color = seshadri[5],   fill = False,label="K-O pairs", histtype = "step", lw = 3, zorder= 2, alpha = 0.7)

plt.axvline(np.sqrt(2) / 2 , 0, 1, label='', ls = ':', color = "firebrick",       lw = 2.5 , zorder = 1)
plt.legend(edgecolor = "b", fancybox = True , fontsize = 16)
plt.tick_params(axis = 'both', labelsize=17)
plt.xlabel("Distance (l.u.)", fontsize = 20)
plt.ylabel("Pair distribution (arb. units)", fontsize = 20)
plt.ylim(0,80)
plt.show() 



In [ ]:
plt.figure(figsize = [8,11])
plt.hist(list_OO , bins = 140 , density = True ,color = seshadri[5], fill = False , label="O-O pairs", histtype = "step", stacked = True, lw = 3.5, zorder= 2, alpha = 0.7)
plt.axvline(np.sqrt(2) / 2 , 0, 1, label='', ls = ':', color = "firebrick",       lw = 2.5 , zorder = 1, alpha = 0.7)
plt.legend(edgecolor = "b", fancybox = True , fontsize = 16)
plt.tick_params(axis = 'both', labelsize=17)
plt.xlabel("Distance (l.u.)", fontsize = 20)
plt.ylabel("Pair distribution (arb. units)", fontsize = 20)
plt.ylim(0,40)
plt.show() 
